# Data Processing and Analysis
Consists of following major components.
1.   Demand calculation
2.   Identifying the price for each product
3. Delivery cost related calculations




In [29]:
import pandas as pd
import numpy as np


outlet_details = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Outlet Details')
sales_details = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Sales Data')
vehicle_data = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Vehicle Details')
inventory_data = pd.read_excel('Sanitized Test Data - 10_10_2024.xlsx', sheet_name='Inventory Details')

In [30]:
print('Number of unique outlets: '+str(len(outlet_details['outlet_id'].unique())))
print('Number of unique products: '+str(len(sales_details['product_id'].unique())))


Number of unique outlets: 675
Number of unique products: 16


Data Cleaning Process

1. Exclude outlet locations that are not typical land areas.
2. 

In [31]:
# Define acceptable ranges for latitude and longitude
LAT_MIN, LAT_MAX = 6.5,7  
LON_MIN, LON_MAX = 79.5, 80.1 

# Filter the dataframe
outlet_details = outlet_details[(outlet_details['gps_latitude'] >= LAT_MIN) & (outlet_details['gps_latitude'] <= LAT_MAX) &
                 (outlet_details['gps_longitude'] >= LON_MIN) & (outlet_details['gps_longitude'] <= LON_MAX)]

valid_outlet_ids = outlet_details['outlet_id'].unique()

# Step 3: Filter the sales_details DataFrame to keep only the rows with valid outlet_ids
sales_details= sales_details[sales_details['outlet_id'].isin(valid_outlet_ids)]


## Demand
#### Current Approach:
Demand Calculation for Each Outlet: In this PoC, demand is calculated for each product at every outlet by assigning the maximum demand observed for each product. This approach helps in estimating the peak demand, accounting variations in product requirements.

#### Possible improvements: 
To enhance the accuracy of demand predictions, it is essential to treat demand fluctuations as a time series forecasting task. This will allow us to capture trends and patterns over time, leading to more reliable and dynamic demand forecasting.



In [32]:
demand_df = sales_details.groupby(['outlet_id', 'product_id'])['qty'].max().reset_index()
demand_df.rename(columns={'qty': 'max_demand'}, inplace=True)
demand_df = demand_df.sort_values(by=['outlet_id'])

In [33]:
demand_2d = demand_df.pivot(index='outlet_id', columns='product_id')
demand_2d = demand_2d.fillna(0)

# Extract the outlet and product mappings
outlet_mapping = {idx: outlet for idx, outlet in enumerate(demand_2d.index)}
product_mapping = {idx: product for idx, product in enumerate(demand_2d.columns)}

# Convert the pivoted DataFrame to a 2D array
demand_array = demand_2d.to_numpy()

# Display the mappings and the array
print("Outlet Mapping:", outlet_mapping)
print("Product Mapping:", product_mapping)
print("Demand Array:\n", demand_array)

Outlet Mapping: {0: 4569, 1: 4604, 2: 5727, 3: 5729, 4: 5801, 5: 5806, 6: 5838, 7: 5903, 8: 5905, 9: 5906, 10: 5907, 11: 5910, 12: 5911, 13: 5912, 14: 5913, 15: 5914, 16: 5917, 17: 5924, 18: 5927, 19: 5930, 20: 5933, 21: 5934, 22: 5944, 23: 5946, 24: 5947, 25: 5950, 26: 5952, 27: 5954, 28: 5956, 29: 5967, 30: 5969, 31: 5970, 32: 5973, 33: 5976, 34: 5979, 35: 5981, 36: 5991, 37: 5993, 38: 5995, 39: 5996, 40: 5997, 41: 5998, 42: 6002, 43: 6005, 44: 6006, 45: 6007, 46: 6008, 47: 6012, 48: 6014, 49: 6015, 50: 6022, 51: 6025, 52: 6031, 53: 6033, 54: 6034, 55: 6035, 56: 6036, 57: 6037, 58: 6038, 59: 6039, 60: 6042, 61: 6043, 62: 6044, 63: 6045, 64: 6049, 65: 6055, 66: 6056, 67: 6057, 68: 6063, 69: 6064, 70: 6068, 71: 6073, 72: 6075, 73: 6076, 74: 6077, 75: 6078, 76: 6079, 77: 6083, 78: 6087, 79: 6090, 80: 6092, 81: 6097, 82: 6098, 83: 6099, 84: 6102, 85: 6103, 86: 6104, 87: 6105, 88: 6109, 89: 6115, 90: 6116, 91: 6119, 92: 6120, 93: 6124, 94: 6129, 95: 6135, 96: 6136, 97: 6138, 98: 6140, 99:

## Per unit price
#### Current Approach:
Demand Calculation for Each Outlet: In this PoC, price per unit for each product is calculated using the most recent sales data. Assuming that the latest price reflects the current market price for each product.
#### Possible improvements: 
Source the price data directly from the distribution center or other reliable sources.



In [34]:
sales_details['invoice_date'] = pd.to_datetime(sales_details['Invoice Date'])

# getting the per item price
sales_details['price'] = sales_details['value'] / sales_details['qty']

sales_details = sales_details.sort_values(by=['product_id', 'Invoice Date'], ascending=[True, False])
latest_price_df = sales_details.drop_duplicates(subset=['product_id'], keep='first')


latest_price_df = latest_price_df[['product_id', 'price']].set_index('product_id')

# order by product_id
latest_price_df = latest_price_df.sort_values(by=['product_id'])

# create a list of price
price_list = latest_price_df['price'].tolist()

## Distance
#### Current Approach:
Haversine formula to calculate the straight-line distance between the distribution center and each outlet. This does not account for actual road conditions, traffic, or routing constraints that might impact real-world travel time or costs.
#### Possible improvements: 
Incorporating actual road distances using a mapping or routing API 



In [35]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Radius of Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Compute differences
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Apply the Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distance in kilometers
    distance = R * c

    return distance

In [36]:
# Pivot point coordinates
pivot_lat, pivot_long = 6.796856130578927, 79.89603028506602

# convert gps_latitude, gps_longitude to float values
outlet_details['gps_latitude'] = outlet_details['gps_latitude'].astype(float)
outlet_details['gps_longitude'] = outlet_details['gps_longitude'].astype(float)

# calculate the distance using haversine function from pivot point to every outlet
outlet_details['distance_from_pivot'] = outlet_details.apply(lambda row: haversine(pivot_lat, pivot_long, row['gps_latitude'], row['gps_longitude']), axis=1)

# keep outlet_id, distance_from_pivot columns
distance = outlet_details[['outlet_id', 'distance_from_pivot']]

# remove duplicate outlet_id
distance = distance.drop_duplicates(subset=['outlet_id'])

# set outlet_id as index
distance = distance.set_index('outlet_id')


# convert the distance_from_pivot column in to a list
distance_array = distance['distance_from_pivot'].tolist()


## Inventoy Cost

Random values are assigned to each product. For an actual scenario sourcing the actual costs are recommended.

In [37]:
# convert the df to a list
inventory_cost = inventory_data['unit_cost'].tolist()

## Priority measures
##### For each outlet priority measure is assigned. In this PoC priority measure is calculated based on the demand volume.
##### Other possible ways: 
- Revenue potential
- Distance from distribution centre

In [38]:
import pandas as pd

# Step 1: Calculate total demand for each outlet
demand_df['total_demand'] = demand_df.groupby('outlet_id')['max_demand'].transform('sum')

# Step 2: Create a new DataFrame with unique outlets and their total demand
PriorityDF = demand_df[['outlet_id', 'total_demand']].drop_duplicates()

# Step 3: Assign priorities based on total demand (higher demand = higher priority)
PriorityDF = PriorityDF.sort_values('total_demand', ascending=False).reset_index(drop=True)
PriorityDF['priority'] = PriorityDF.index + 1

# Normalize the priority column to a range between 0 and 1, with 1 being the highest priority
PriorityDF['normalized_priority'] = 1 - (PriorityDF['priority'] - PriorityDF['priority'].min()) / (PriorityDF['priority'].max() - PriorityDF['priority'].min())

print(PriorityDF)


     outlet_id  total_demand  priority  normalized_priority
0      1006537          1283         1             1.000000
1         6063           824         2             0.998480
2       684084           483         3             0.996960
3         6115           355         4             0.995441
4       260508           342         5             0.993921
..         ...           ...       ...                  ...
654     269509             1       655             0.006079
655       6014             1       656             0.004559
656     263466             1       657             0.003040
657     265010             0       658             0.001520
658     253721             0       659             0.000000

[659 rows x 4 columns]


# Optimization Problem

In [39]:
# !python /Applications/CPLEX_Studio2211/python/setup.py install

In [40]:
import docplex.mp
# first import the Model class from docplex.mp
from docplex.mp.model import Model

# create one model instance, with a name
model = Model(name='Revenue_Maximization')

In [41]:
# by default, all variables in Docplex have a lower bound of 0 and infinite upper bound
# y[j]: Binary variable indicating if shop j is reached
y = model.binary_var_list(keys=675, name="y")
#

# q[i, j]: Quantity of product i delivered to shop j as an integer variable
q = model.integer_var_matrix(keys1=16, keys2=675, name="q",lb=0)

In [42]:
# random data generation
Q = 100000
n = len(outlet_details['outlet_id'].unique()) # number of stores
m = len(sales_details['product_id'].unique()) # number of products

D = demand_array
# D[0] = [0]*16

Tmax = 5

IC = inventory_cost 
# C = np.random.randint(10, 51, size=n)
C = distance_array

# A = np.random.randint(20, 151, size=(m, n))
A = price_list
Pr = np.random.randint(1, 6, size=n)

In [43]:
D[0][0]

np.float64(0.0)

In [44]:
# Vehicle Capacity Constraint
## for each outlet the sum of q has to be less than the total capacity of vehicles served for that outlet // change this part
model.add_constraint(model.sum(q)<= Q, "total_qty_constraint")

# # Qty Satisfaction Constraint
for i in range(m): # i: product
    for j in range(n): # j: store
        model.add_constraint(q[i, j] <= Q*y[j], f"demand_constraint_{i}_{j}") # store -> product

# # Demand Satisfaction Constraint
for i in range(m): # i: product
    for j in range(n): # j: store
        model.add_constraint(q[i, j] <= D[j][i], f"demand_constraint_{i}_{j}") # store -> product


In [45]:
model

docplex.mp.Model['Revenue_Maximization']

In [46]:
# Objective function components
revenue = model.sum(A[i] * Pr[j] * q[i, j] for i in range(m) for j in range(n))
delivery_costs = model.sum(C[j] * y[j] for j in range(n))
inventory_cost = model.sum(IC[i]*q[i, j] for i in range(m) for j in range(n))

Z = revenue - delivery_costs-inventory_cost
# Define objective function
model.maximize(Z)


In [47]:
model.print_information()

Model: Revenue_Maximization
 - number of variables: 11475
   - binary=675, integer=10800, continuous=0
 - number of constraints: 19771
   - linear=19771
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP


In [48]:
s = model.solve()
model.print_solution()

objective: 46349724.671
status: OPTIMAL_SOLUTION(2)
  y_0=1
  y_1=1
  y_2=1
  y_3=1
  y_4=1
  y_5=1
  y_6=1
  y_7=1
  y_8=1
  y_9=1
  y_10=1
  y_11=1
  y_12=1
  y_13=1
  y_14=1
  y_15=1
  y_16=1
  y_17=1
  y_18=1
  y_19=1
  y_20=1
  y_21=1
  y_22=1
  y_23=1
  y_24=1
  y_25=1
  y_26=1
  y_27=1
  y_28=1
  y_29=1
  y_30=1
  y_31=1
  y_32=1
  y_33=1
  y_34=1
  y_35=1
  y_36=1
  y_37=1
  y_38=1
  y_39=1
  y_40=1
  y_41=1
  y_42=1
  y_43=1
  y_44=1
  y_45=1
  y_46=1
  y_47=1
  y_48=1
  y_49=1
  y_50=1
  y_51=1
  y_52=1
  y_53=1
  y_54=1
  y_55=1
  y_56=1
  y_57=1
  y_58=1
  y_59=1
  y_60=1
  y_61=1
  y_62=1
  y_63=1
  y_64=1
  y_65=1
  y_66=1
  y_67=1
  y_68=1
  y_69=1
  y_70=1
  y_71=1
  y_72=1
  y_73=1
  y_74=1
  y_75=1
  y_76=1
  y_77=1
  y_78=1
  y_79=1
  y_80=1
  y_81=1
  y_82=1
  y_83=1
  y_84=1
  y_85=1
  y_86=1
  y_87=1
  y_88=1
  y_89=1
  y_90=1
  y_91=1
  y_92=1
  y_93=1
  y_94=1
  y_95=1
  y_96=1
  y_97=1
  y_98=1
  y_99=1
  y_100=1
  y_101=1
  y_102=1
  y_103=1
  y_104=1
  y_105=

In [51]:
import numpy as np

# Define the dimensions
n =  len(outlet_details['outlet_id'].unique()) # Number of outlets
m = len(sales_details['product_id'].unique())   # Number of products

# Initialize a NumPy array filled with zeros
outlets_products = np.zeros((n, m), dtype=int)

# Assuming s.iter_variables() returns an iterable of variable objects
for var in s.iter_variables():
    if var.name.startswith('q_'):
        outlet_number = int(var.name.split('_')[2])  # Extract outlet number
        product_id = int(var.name.split('_')[1])      # Extract product ID
        product_qty = int(var.solution_value)          # Get product quantity
        
        # Add the quantity to the corresponding position in the NumPy array
        outlets_products[outlet_number][product_id] += product_qty

print(outlets_products)

[[0 2 2 ... 2 0 0]
 [0 2 1 ... 2 0 0]
 [0 2 2 ... 0 0 0]
 ...
 [0 4 5 ... 2 2 0]
 [0 1 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]]


In [52]:
outlet_list = sorted(int(i) for i in sales_details['outlet_id'].unique())
product_list = sorted(int(i) for i in sales_details['product_id'].unique())
df = pd.DataFrame(outlets_products)
df.columns = product_list
df.index = outlet_list


In [53]:
df

,4592,5064,5065,5066,5068,5069,5070,5072,5073,5076,5077,5078,5079,5470,5471
4569,0,2,2,0,0,2,2,2,0,0,2,2,2,0,0
4604,0,2,1,0,0,2,0,1,0,2,1,2,2,0,0
5727,0,2,2,0,0,0,0,0,0,0,1,0,0,0,0
5729,0,1,1,0,0,2,1,0,0,1,2,2,2,0,0
5801,0,2,0,0,2,3,3,2,0,2,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033680,0,1,2,0,0,1,2,2,0,0,0,1,1,0,0
1033681,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0
1033682,0,4,5,0,99,2,3,7,0,1,1,1,2,2,0
1033683,0,1,0,0,0,2,0,1,0,2,1,2,1,0,0


In [54]:
import pandas as pd
import plotly.express as px



# Create the map
fig = px.scatter_mapbox(
    outlet_details,
    lat="gps_latitude",
    lon="gps_longitude",
    hover_name="outlet_id",  # Show outlet ID when hovering over points
    zoom=10,  # Initial map zoom level
    mapbox_style="carto-positron"  # Map style for a clean background
)

# Center the map around the average latitude and longitude values for better visualization
fig.update_layout(
    title="Outlet Locations Map",
    mapbox=dict(center=dict(lat=6.80423464, lon=79.95780262), zoom=12)
)

# Show the plot
fig.show()

In [55]:
# Create a click event for the outlets
def get_product_quantities(outlet_id):
    return df.loc[outlet_id].tolist()

In [56]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='outlet-map', figure=fig),
    dcc.Graph(id='product-quantities')
])

@app.callback(
    Output('product-quantities', 'figure'),
    Input('outlet-map', 'clickData')
)
def display_product_quantities(clickData):
    if clickData is None:
        return go.Figure()  # Empty figure if no click
    outlet_id = clickData['points'][0]['hovertext']  # Get the outlet ID
    quantities = get_product_quantities(outlet_id)
    # Create a bar chart for product quantities
    
    data = pd.DataFrame({'Product': df.columns, 'Quantity': quantities})
    non_zero_data = data[data['Quantity'] > 0]
    # Create a bar chart for product quantities
    product_fig = go.Figure(data=[go.Bar(x=df.columns, y=quantities)])
    product_fig.update_layout(
        title=f'Product Quantities for Outlet {outlet_id}',
        xaxis=dict(
            tickmode='linear',
            dtick=1,
            range=[non_zero_data['Product'].iloc[0], non_zero_data['Product'].iloc[-1]]
        )
    )
    return product_fig

if __name__ == '__main__':
    app.run_server(debug=True)


In [57]:
print(y[0].solution_value)

1.0


In [58]:
import folium
from folium import map
import numpy as np
from docplex.mp.model import Model


# Function to adjust costs based on environmental factors
def adjust_costs(cost_matrix, environmental_data):
    adjusted_matrix = cost_matrix.copy()
    for (i, j), factor in environmental_data.items():
        adjusted_matrix[i][j] *= factor
        adjusted_matrix[j][i] *= factor
    return adjusted_matrix

# Function to create a distance matrix using Haversine distance
def haversine_distance(coord1, coord2):
    R = 6371.0  # Earth radius in kilometers
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# def calculate_distance_matrix(locations):
#     num_locations = len(locations)
#     distance_matrix = np.zeros((num_locations, num_locations))
#     for i in range(num_locations):
#         for j in range(num_locations):
#             if i != j:
#                 distance_matrix[i][j] = haversine_distance(locations[i], locations[j])
#     return distance_matrix

def calculate_distance_matrix(locations):
    """
    Calculate the distance matrix for locations with (outlet_id, latitude, longitude).

    Parameters:
    locations (list of tuples): A list of tuples in the format (outlet_id, latitude, longitude).

    Returns:
    numpy.ndarray: A 2D array representing the pairwise Haversine distances.
    """
    num_locations = len(locations)
    distance_matrix = np.zeros((num_locations, num_locations))

    for i in range(num_locations):
        for j in range(num_locations):
            if i != j:
                lat_lon_i = (locations[i][1], locations[i][2])  # Extract (latitude, longitude)
                lat_lon_j = (locations[j][1], locations[j][2])  # Extract (latitude, longitude)
                distance_matrix[i][j] = haversine_distance(lat_lon_i, lat_lon_j)

    return distance_matrix


# Function to solve TSP
def solve_tsp(locations, environmental_data, start_index=0):
    mdl = Model(name="TSP_with_environmental_factors")
    distance_matrix = calculate_distance_matrix(locations)
    adjusted_matrix = adjust_costs(distance_matrix, environmental_data)

    num_locations = len(locations)
    x = mdl.binary_var_matrix(num_locations, num_locations, name="x")
    mdl.minimize(mdl.sum(adjusted_matrix[i][j] * x[i, j] for i in range(num_locations) for j in range(num_locations) if i != j))

    for i in range(num_locations):
        mdl.add_constraint(mdl.sum(x[i, j] for j in range(num_locations) if j != i) == 1, f"depart_{i}")
        mdl.add_constraint(mdl.sum(x[j, i] for j in range(num_locations) if j != i) == 1, f"arrive_{i}")

    mdl.add_constraint(mdl.sum(x[start_index, j] for j in range(num_locations) if j != start_index) == 1)
    mdl.add_constraint(mdl.sum(x[i, start_index] for i in range(num_locations) if i != start_index) == 1)

    u = mdl.integer_var_list(num_locations, lb=0, ub=num_locations - 1, name="u")
    for i in range(1, num_locations):
        for j in range(1, num_locations):
            if i != j:
                mdl.add_constraint(u[i] - u[j] + num_locations * x[i, j] <= num_locations - 1)
    mdl.context.cplex_parameters.timelimit = 300  # Time limit in seconds (5 minutes)
    mdl.context.cplex_parameters.threads = 12
    solution = mdl.solve(log_output=True)
    route = [(i, j) for i in range(num_locations) for j in range(num_locations) if i != j and x[i, j].solution_value > 0.5]
    return route, locations

# Plotting function


def plot_route_on_map(route, locations, start_location, locations_out):
    """
    Plot the TSP route on a folium map with outlet IDs displayed on markers.

    Parameters:
    - route: List of tuples indicating the TSP route (e.g., [(0, 1), (1, 2)]).
    - locations: List of tuples in the format (outlet_id, latitude, longitude).
    - start_location: Tuple (latitude, longitude) for the map's starting point.

    Returns:
    - folium.Map object.
    """
    # Initialize the map centered at the starting location
    map_ = folium.Map(location=start_location, zoom_start=12)

    total_revenue = 0

    for outlet_id, lat, lon in locations_out:
        folium.Marker(
                location=(lat, lon),
                tooltip="Unvisited Shop",
                icon=folium.Icon(color="orange")
            ).add_to(map_)

    # Add markers for all locations
    for outlet_id, lat, lon in locations:
        if outlet_id == 101:
            folium.Marker(
                location=(lat, lon),
                tooltip="Distribution Center",
                icon=folium.Icon(color="red")
            ).add_to(map_)
            continue
            
        # Get the quantities for this outlet
        quantities = df.loc[outlet_id].values
        
        # Create the tooltip content
        tooltip_content = f"Outlet ID: {outlet_id}<br>Lat: {lat:.4f}, Lon: {lon:.4f}<br>"
        for product_id, quantity in zip(df.columns, quantities):
            if quantity > 0:  # Only display product if quantity is non-zero
                tooltip_content += f"Product {product_id} : {quantity}<br>"
                total_revenue += quantity * price_list[next((k for k, v in product_mapping.items() if v == ('max_demand', product_id)), None)]
        
        # Add the marker with the tooltip
        folium.Marker(
            location=(lat, lon),
            tooltip=tooltip_content
        ).add_to(map_)
    # Add route connections using the route information
    total_distance = 0
    for i, j in route:
        start = (locations[i][1], locations[i][2])  # Extract (latitude, longitude)
        end = (locations[j][1], locations[j][2])    # Extract (latitude, longitude)
        folium.PolyLine(
            locations=[start, end],
            color="blue",
            weight=2.5,
            opacity=1
        ).add_to(map_)
        total_distance += haversine_distance(start, end)

    # Add summary information
    summary_html = f"""
    <div style="
        position: fixed;
        top: 50px; right: 50px;
        z-index: 1000;
        background-color: white;
        padding: 10px;
        border: 2px solid black;
        border-radius: 5px;
        box-shadow: 5px 5px 15px rgba(0,0,0,0.3);
        font-size: 14px;">
        <b>Route Summary</b><br>
        Start Location: {start_location[0]:.4f}, {start_location[1]:.4f}<br>
        Number of Stops: {len(locations)}<br>
        Total Distance: {total_distance:.2f} km<br>
        Total Revenue: {total_revenue:,.2f} LKR
    </div>
    """
    summary_element = map.Element(summary_html)
    map_.get_root().html.add_child(summary_element)

    return map_


# Main execution
if __name__ == "__main__":
    beat = 8
    # Replace with your actual location data
    locations_full = outlet_details[outlet_details['beat'] == beat][['outlet_id','gps_latitude', 'gps_longitude']].values.tolist()

    locations_in = list(filter(lambda loc: y[next((k for k, v in outlet_mapping.items() if v == loc[0]), None)].solution_value == 1, locations_full))
    locations_out = list(filter(lambda loc: y[next((k for k, v in outlet_mapping.items() if v == loc[0]), None)].solution_value == 0, locations_full))

    locations_in.insert(0, [101, pivot_lat, pivot_long])

    # print(locations_in)
    environmental_data = {}
    distance_matrix = calculate_distance_matrix(locations_in)
    route, locations = solve_tsp(locations_in, environmental_data)
    start_location = (locations[0][1], locations[0][2])  # Extract latitude and longitude
    map_ = plot_route_on_map(route, locations, start_location, locations_out)
    map_.save(f"route_beat_{beat}.html")



Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 12
CPXPARAM_TimeLimit                               300
Tried aggregator 1 time.
MIP Presolve eliminated 2 rows and 46 columns.
Reduced MIP has 1982 rows, 2024 columns, and 9636 nonzeros.
Reduced MIP has 1980 binaries, 44 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (5.34 ticks)
Probing time = 0.01 sec. (3.46 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 1982 rows, 2024 columns, and 9636 nonzeros.
Reduced MIP has 1980 binaries, 44 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.98 ticks)
Probing time = 0.01 sec. (3.45 ticks)
Clique table members: 1030.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solution time = 0.01 sec. (3.23 ticks)

        Nodes                   